In [60]:

from fastapi import APIRouter
from supabase import create_client, Client
from config import SUPABASE_URL, SUPABASE_KEY, PODCAST_API_KEY, PODCAST_USER_ID, PODCAST_API_URL
import requests

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

In [61]:
sample_headers = {
    "Content-Type": "application/json",
    "X-USER-ID": PODCAST_USER_ID,
    "X-API-KEY": PODCAST_API_KEY
}

In [62]:
def get_podcast_data(podcast_name = "Lex Fridman"):
    podcast_request_body = f"""{{
        getPodcastSeries(name: "Lex Fridman") {{
        uuid
        name
        authorName
        imageUrl
        description
        episodes(page:5) {{
            uuid
            datePublished
            name
            description
            duration
            imageUrl
            audioUrl
        }}
        genres
    }}
    }}
    """
    json_request = {"query": podcast_request_body}
    res = requests.post(PODCAST_API_URL, headers=sample_headers, json=json_request)
    return res.json()

In [63]:
res = get_podcast_data()

In [64]:
data = [res['data']['getPodcastSeries']['episodes'][3]]
data

[{'uuid': 'eb0a3ee3-46ea-487a-9250-3b166b08181c',
  'datePublished': 1699556626,
  'name': '#400 – Elon Musk: War, AI, Aliens, Politics, Physics, Video Games, and Humanity',
  'description': '<p>Elon Musk is CEO of X, xAI, SpaceX, Tesla, Neuralink, and The Boring Company.</p>\n<p>Thank you for listening ❤ Please support this podcast by checking out our sponsors:<br />\n&#8211; LMNT: <a href="https://drinkLMNT.com/lex">https://drinkLMNT.com/lex</a> to get free sample pack<br />\n&#8211; Eight Sleep: <a href="https://www.eightsleep.com/lex">https://www.eightsleep.com/lex</a> to get special savings<br />\n&#8211; BetterHelp: <a href="https://betterhelp.com/lex">https://betterhelp.com/lex</a> to get 10% off<br />\n&#8211; SimpliSafe: <a href="https://simplisafe.com/lex">https://simplisafe.com/lex</a> to get free security camera plus 20% off<br />\n&#8211; Shopify: <a href="https://shopify.com/lex">https://shopify.com/lex</a> to get $1 per month trial<br />\n&#8211; NetSuite: <a href="http:

In [65]:
from datetime import datetime

def insert_podcast(data):
    return (
        supabase.table("podcasts").insert({
        "title": data['name'],
        "author_name": data['authorName'],
        "image_url": data['imageUrl'],
        "description": data['description'],
    }).execute())

def insert_episodes(data):
    return (supabase.table("episodes").insert([{
        "title": episode['name'],
        "description": episode['description'],
        "duration": episode['duration'],
        "image_url": episode['imageUrl'],
        "created_at": datetime.fromtimestamp(episode['datePublished']).isoformat(),
        "audio_url": episode['audioUrl'],
    } for episode in data]).execute())

In [66]:
# response = insert_podcast(res['data']['getPodcastSeries'])

In [67]:
episode_response = insert_episodes(res['data']['getPodcastSeries']['episodes'])

In [68]:
episode_response.data

[{'id': 51,
  'title': '#403 – Lisa Randall: Dark Matter, Theoretical Physics, and Extinction Events',
  'description': '<p>Lisa Randall is a theoretical physicist at Harvard. Please support this podcast by checking out our sponsors:<br />\n&#8211; <b>Babbel</b>: <a href="https://babbel.com/lexpod">https://babbel.com/lexpod</a> and use code Lexpod to get 55% off<br />\n&#8211; <b>Notion</b>: <a href="https://notion.com">https://notion.com</a><br />\n&#8211; <b>SimpliSafe</b>: <a href="https://simplisafe.com/lex">https://simplisafe.com/lex</a> to get free security camera plus 20% off<br />\n&#8211; <b>LMNT</b>: <a href="https://drinkLMNT.com/lex">https://drinkLMNT.com/lex</a> to get free sample pack<br />\n&#8211; <b>InsideTracker</b>: <a href="https://insidetracker.com/lex">https://insidetracker.com/lex</a> to get 20% off</p>\n<p>Transcript: <a href="https://lexfridman.com/lisa-randall-transcript">https://lexfridman.com/lisa-randall-transcript</a></p>\n<p><b>EPISODE LINKS:</b><br />\nL

In [69]:
episode_ids =[episode['id'] for episode in episode_response.data]
episode_ids

[51, 52, 53, 54, 55, 56, 57, 58, 59, 60]

In [70]:
podcast_id = 4

In [71]:
def insert_podcast_episodes(podcast_id, episode_ids):
    return (supabase.table("episode_podcast").insert([{
        "podcast_id": podcast_id,
        "episode_id": episode_id
    } for episode_id in episode_ids]).execute())

In [72]:
relationship_response = insert_podcast_episodes(podcast_id, episode_ids)